### Document
- [awesome-devblog : feeds](https://awesome-devblog.now.sh/api/korean/people/feeds)

In [4]:
# %%writefile ./document.py
import requests, json
import os, re, csv
import pandas as pd
import numpy as np
from enum import Enum
from tqdm import trange

class KEYS(Enum):
    # -1 : 아직 라벨링 안함 (default)
    # 0  : 개발과 관련없는 문서
    # 1  : 개발과 관련있는 문서
    LABEL = 'label'
    
    # TAGS + TITLE + DESC
    TEXT = 'text'
    
    # DATA_URL 결과 파싱용 Keys(Beans)
    ID = '_id'
    TITLE = 'title'
    DESC = 'description'
    TAGS = 'tags'
    LINK = 'link'
    
    def getDocKeys():
        return [KEYS.ID.value, KEYS.TITLE.value, KEYS.DESC.value, KEYS.TAGS.value, KEYS.LINK.value]
    
    def getTitleBlackList():
        return ['', 'about']
    
    def getTextKeys():
        return [KEYS.TAGS.value, KEYS.TITLE.value, KEYS.DESC.value]

class Document():
    
    def __init__(self, load=True, debug=False):
        self.debug = debug
        
        # Constant
        self.DATA_URL = 'https://awesome-devblog.now.sh/api/korean/people/feeds'
        self.DOCUMENTS_PATH = './data/documents.csv'
        self.MAX_REQ_SIZE = 5000
        
        if load:
            if debug: print('> Updating documents...')
            self._updateDocs()

            if debug: print('> Loading documents...')
            self.data = self._getDocs()

            if debug: print('> Done!')
        
    def _getTotal(self):
        """
        전체 문서 개수 요청
        """
        res = requests.get(self.DATA_URL, { 'size': 1 })
        res.raise_for_status()
        doc = res.json()
        return doc['total'][0]['count']

    def _reqDoc(self, page, size, preprocessing=False):
        """
        문서 요청
        - page는 0 부터 시작
        - 전처리(self._preprocessing) 후 반환
        """
        page += 1
        params = {
            'sort': 'date.asc',
            'page': page,
            'size': size
        }
        res = requests.get(self.DATA_URL, params)
        res.raise_for_status()
        doc = res.json()
        
        # json to dataframe
        doc = pd.DataFrame(doc['data'], columns=KEYS.getDocKeys())
        
        # add label
        doc.insert(0, KEYS.LABEL.value, -1)
        
        if preprocessing:
            return self._preprocessing(doc)
        else:
            return doc
    
    def _preprocessing(self, doc):
        """
        문서 전처리
        - KEYS 이외의 key 삭제
        - [tag] list join to string
        - [title / description / tags] 영어, 한글, 공백 이외의 것들 모두 삭제
        - \ 삭제
        - 2번 이상의 공백 1개로 통합
        - 영어 대문자 소문자로 변환
        - 앞뒤 공백 삭제
        """
        # tags
        doc[KEYS.TAGS.value] = doc[KEYS.TAGS.value].apply(lambda x: ' '.join(x))
            
        # title, description, tags
        for key in KEYS.getTextKeys():
            doc[key] = doc[key].apply(lambda x: re.sub('[^가-힣a-zA-Z\s]', '', repr(x)))

        # all
        def preprocessing(x):
            text = re.sub(r'\\', '', repr(x))
            x = re.sub('\s{2,}', ' ', x)
            x = x.lower()
            x = x.strip()
            return x

        for key in KEYS.getDocKeys():
            doc[key] = doc[key].apply(preprocessing)

        # remove blacklist
        doc.drop(doc[doc[KEYS.TITLE.value].isin(KEYS.getTitleBlackList())].index).reset_index()
                        
        # create text column
        join_with = lambda x: ' '.join(x.dropna().astype(str))
        doc[KEYS.TEXT.value] = doc[KEYS.getTextKeys()].apply(
            join_with,
            axis=1
        )
        return doc
        

    def _reqDocs(self, size, start_page=0):
        """
        전체 문서 요청
        """
        total = self._getTotal()
        if size > self.MAX_REQ_SIZE: size = self.MAX_REQ_SIZE
        total_req = round(total/size + 0.5)
        docs = pd.DataFrame()
        for i in trange(start_page, total_req):
            doc = self._reqDoc(i, size)
            if docs.empty:
                docs = doc
            else:
                docs = docs.append(doc)
        return self._preprocessing(docs)

    def _updateDocs(self):
        """
        최신 문서 추가
        - 데이터가 없는 경우, 전체 데이터를 가져옴
        - 기존 데이터가 있는 경우, 없는 데이터만 추가
        """
        size = self.MAX_REQ_SIZE
        
        if not os.path.isfile(self.DOCUMENTS_PATH):
            # 데이터가 없는 경우
            docs = self._reqDocs(size)
            docs.to_csv(self.DOCUMENTS_PATH, sep=",", index=False)
        else:
            # 기존 데이터가 있는 경우
            num_new_docs = 0
            docs = pd.read_csv(self.DOCUMENTS_PATH, delimiter=',')
            total = self._getTotal()
            total_docs = len(docs)
            new_docs_num = total - total_docs
            new_docs = self._reqDocs(size, total_docs // size)
            
            # _id가 기존 데이터에 존재하지 않는 경우에만 추가
            docs.append(new_docs[~new_docs[KEYS.ID.value].isin(docs[KEYS.ID.value])])
            docs.to_csv(self.DOCUMENTS_PATH, sep=",", index=False)
            if self.debug:
                if total_docs == len(docs):
                    print('문서가 최신입니다.')
                else:
                    print(f'신규 문서 {num_new_docs}개 추가')

    def _getDocs(self):
        """
        전체 문서 조회
        """
        if not os.path.isfile(self.DOCUMENTS_PATH):
            raise FileNotFoundError(f'The scripts file {self.DOCUMENTS_PATH} does not exist')
        return pd.read_csv(self.DOCUMENTS_PATH, delimiter=',', dtype={KEYS.LABEL.value: np.int64})
    
    def syncDocLabel(self, old_document_path, sep, override=False):
        """
        기존 라벨링한 데이터를 신규 문서에 반영
        """
        def preprocessing(text):
            text = re.sub(r'\\', '', text)
            text = re.sub('\s{2,}', ' ', text)
            text = text.lower()
            text = text.strip()
            return text
        
        document = pd.read_csv(self.DOCUMENTS_PATH, delimiter=',')
        old_document = pd.read_csv(old_document_path, delimiter=sep)
        for index, row in old_document[:10000].iterrows():
            link = preprocessing(row.link)
            title = re.sub('[^가-힣a-zA-Z\s]', '', row.title)
            title = preprocessing(title)
            
            label = int(row.label)
            if not len(document.loc[document.title.str.strip() == title.strip()]) and not len(document.loc[document.link == link]):
                print(f'not found : {row.title}')
            elif len(document.loc[document.title.str.strip() == title.strip()]):
                document.loc[document.title.str.strip() == title.strip(), KEYS.LABEL.value] = label
            elif len(document.loc[document.link == link]):
                document.loc[document.link == link, KEYS.LABEL.value] = label
        
        # save synchronized document
        if override:
            document.to_csv(self.DOCUMENTS_PATH, sep=",", index=False)
        print('done')

Overwriting ./document.py


In [2]:
doc = Document(True, True)

> Updating documents...


100%|██████████| 8/8 [00:21<00:00,  2.67s/it]


> Loading documents...
> Done!


In [3]:
doc.syncDocLabel('data/labeled.tsv', '\t', True)

not found : [Today's CCM] 다시 한번
not found : [Today's CCM] 주님만이 모든 것
not found : [Today's CCM]성령님
not found : [광명/철산 맛집] 반올림피자샵 - 배달피자 맛집
not found : [공주 부여 여행 1일차] 마곡사 근처 식사 & 맛집 바람처럼~ 구름처럼~ (더덕정식, 도토리묵, 명품파전)
not found : [광명/철산 맛집] 448돈까스 - 숙성생등심돈까스
not found : 남양읍맛집 이춘봉인생치킨 남양치킨 인정!
not found : 경기 화성 비봉 맛집 / 맛골칼국수 다 맛있네요~(코다리찜)
done
